In [197]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import numpy as np

my_data = pd.read_csv('train.csv')

# Тестовое задание - Елькин СВ

### Да, я еще удивился, как это вообще возможно было) 

## Что мы имеем и что можно предположить
#### Я предположу, что можно построить модель линейной регрессии - предположить значение параметра bugs по введенным параметрам
**Сложности** <br>
1 - почти все параметры - категориальные признаки <br>
2 - признаков несколько - какие выбрать

## Каков план
Я буду тестировать гипотезу, что на количество багов могут влиять почти все факторы, за исключением хэша и комментария<br>
Первым делом применим встроеный метод one-hot encoding.

In [198]:
my_data

,repository_name,commit_hash,commit_date,commit_author,commit_message,bugs
0,mlm,b28abb42,2020-06-12T14:26:12,Victor,объяснения предсказаний с помощью lime,4
1,mlm,438b5c4d,2020-06-12T11:53:10,Victor,Поставил lime вместо eli5,3
2,mlm,8e6e4f31,2020-06-11T15:19:03,Victor,Скрипт для создния пакетов обновления,4
3,mlm,5e0956a6,2020-06-11T10:42:29,Victor,Сортировка весов по убыванию,3
4,mlm,b9f4b40c,2020-06-11T09:25:19,Victor,не надо конвертировать массив до операции по с...,7
...,...,...,...,...,...,...
314,sensor,5b1ecbcd,2020-03-23T12:54:35,Dabe,setting hostname,1
315,sensor,a3f5b753,2020-03-23T12:30:52,Dabe,heralding default dockerfile,2
316,sensor,e8155942,2020-03-23T11:27:40,Dabe,unnesserity /data path permissions setting,3
317,sensor,e748af51,2020-03-23T11:20:50,Dabe,added trapconductor,1


#### Что делал с временем коммита:<br>
1) Выделим отдельно дни недели<br>
2) Выделим отдельно месяца<br>
3) Из времени (час-минуты) возьмем только часы и отнормируем (через стандартное нормальное)


In [199]:
import matplotlib.dates as dates
new_x = dates.datestr2num(my_data['commit_date'])

In [200]:
new_df = pd.to_datetime(my_data['commit_date'])
new_df = new_df.rename('day')

In [201]:
new_df = new_df.apply(lambda x: x.strftime('%A'))
my_data = pd.concat([my_data, new_df], axis=1)

In [202]:
new_df = pd.to_datetime(my_data['commit_date'])
new_df = new_df.rename('month')

In [203]:
new_df = new_df.apply(lambda x: x.month_name())
my_data = pd.concat([my_data, new_df], axis=1)

In [204]:
new_df = pd.to_datetime(my_data['commit_date'])
new_df = new_df.rename('am_pm')

In [205]:
new_df = new_df.apply(lambda x: x.hour)
new_df = (new_df - new_df.mean())/new_df.std()
my_data = pd.concat([my_data, new_df], axis=1)

In [206]:
my_data

,repository_name,commit_hash,commit_date,commit_author,commit_message,bugs,day,month,am_pm
0,mlm,b28abb42,2020-06-12T14:26:12,Victor,объяснения предсказаний с помощью lime,4,Friday,June,0.189921
1,mlm,438b5c4d,2020-06-12T11:53:10,Victor,Поставил lime вместо eli5,3,Friday,June,-0.814246
2,mlm,8e6e4f31,2020-06-11T15:19:03,Victor,Скрипт для создния пакетов обновления,4,Thursday,June,0.524643
3,mlm,5e0956a6,2020-06-11T10:42:29,Victor,Сортировка весов по убыванию,3,Thursday,June,-1.148968
4,mlm,b9f4b40c,2020-06-11T09:25:19,Victor,не надо конвертировать массив до операции по с...,7,Thursday,June,-1.483690
...,...,...,...,...,...,...,...,...,...
314,sensor,5b1ecbcd,2020-03-23T12:54:35,Dabe,setting hostname,1,Monday,March,-0.479523
315,sensor,a3f5b753,2020-03-23T12:30:52,Dabe,heralding default dockerfile,2,Monday,March,-0.479523
316,sensor,e8155942,2020-03-23T11:27:40,Dabe,unnesserity /data path permissions setting,3,Monday,March,-0.814246
317,sensor,e748af51,2020-03-23T11:20:50,Dabe,added trapconductor,1,Monday,March,-0.814246


### Тут делаем one-hot

In [207]:
from sklearn.preprocessing import LabelBinarizer

In [208]:
repository_to_vector = LabelBinarizer()
repository_to_vector.fit(my_data['repository_name'])
transformed = repository_to_vector.transform(my_data['repository_name'])
new_df = pd.DataFrame(transformed)
my_data = pd.concat([my_data, new_df], axis=1).drop(['repository_name'], axis=1)


In [209]:
commit_to_vector = LabelBinarizer()
commit_to_vector.fit(my_data['commit_author'])
transformed = commit_to_vector.transform(my_data['commit_author'])
new_df = pd.DataFrame(transformed)
my_data = pd.concat([my_data, new_df], axis=1).drop(['commit_author'], axis=1)

In [210]:
commit_to_vector = LabelBinarizer()
commit_to_vector.fit(my_data['day'])
transformed = commit_to_vector.transform(my_data['day'])
new_df = pd.DataFrame(transformed)
my_data = pd.concat([my_data, new_df], axis=1).drop(['day'], axis=1)

In [211]:
commit_to_vector = LabelBinarizer()
commit_to_vector.fit(my_data['month'])
transformed = commit_to_vector.transform(my_data['month'])
new_df = pd.DataFrame(transformed)
my_data = pd.concat([my_data, new_df], axis=1).drop(['month'], axis=1)

In [212]:
my_data = my_data.drop(['commit_hash','commit_date','commit_message'], axis=1)

In [213]:
my_data

,bugs,am_pm,0,1,2,3,4,5,0,1,...,1,2,3,4,5,6,0,1,2,3
0,4,0.189921,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,3,-0.814246,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,4,0.524643,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
3,3,-1.148968,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
4,7,-1.483690,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
314,1,-0.479523,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
315,2,-0.479523,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
316,3,-0.814246,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
317,1,-0.814246,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0


## Попробуем линейную регрессию 

In [214]:
from sklearn.linear_model import LinearRegression

In [215]:
model = LinearRegression(fit_intercept=True)

**отнормируем данные по колонке bugs**

In [216]:
checker = (my_data['bugs'] - my_data['bugs'].mean())/my_data['bugs'].std()

In [217]:
my_data =my_data.drop(['bugs'], axis=1)

In [218]:
train_data = my_data[:-20]
train_label = checker[:-20]
test_data = my_data[-20:]
test_label = checker[-20:]

In [219]:
model.fit(train_data, train_label,)

LinearRegression()

In [220]:
predicted_data1 = model.predict(test_data)
predicted_data2 = model.predict(train_data)

## Посмотрим, что получилось  

#### В этот раз мерой ошибки будет Mean Absolute Percentage Error (MAPE)  

In [221]:
from sklearn.metrics import mean_absolute_percentage_error

In [222]:
mean_absolute_percentage_error(test_label, predicted_data1)

81.4837737569936

In [223]:
mean_absolute_percentage_error(train_label, predicted_data2)

53.23986149908028

## Итог 
Линейная регрессия проявлеят себя не лучшим образом для данной задачи <br>
Ошибка на тренировочном - 53.2, на валидационном - 81.4, это не хорошо

## Тогда применим прогностическую модель на xgboost

In [224]:
import xgboost as xgb

**Добавим имена one-hot колонкам**

In [225]:
test_data.columns =[item for item in range(0, len(test_data.columns))]
train_data.columns =  [item for item in range(0, len(test_data.columns))]

In [226]:
boost_train = xgb.DMatrix(train_data, train_label)
boost_test = xgb.DMatrix(test_data)

**Зададим параметры**<br>
будем использовать дефолтный параметр **леса** бустера и дефолтный параметр **squarederror**

In [227]:
params = {'objective': 'reg:squarederror'}

**прогоняем на кросс-валидации с метрикой rmse**

In [228]:
cv = xgb.cv(params, dtrain, metrics = ('rmse'), verbose_eval=False, nfold=10, show_stdv=False, num_boost_round=1000)


**обучаем xgboost с оптимальным числом деревьев, подобранным на кросс-валидации**

In [229]:
bst = xgb.train(params, dtrain, num_boost_round=cv['test-rmse-mean'].argmin())
prediction_train = bst.predict(dtrain)
prediction_test = bst.predict(dtest)

In [230]:
mean_absolute_percentage_error(train_label, prediction_train)

29.882002849337823

In [231]:
mean_absolute_percentage_error(test_label, prediction_test)

15.350783108208987

### Итог
#### Ошибка на тренировочном - 29.8, на валидационном - 15.3, что гораздо лучше в сравнении с линейной регрессией.

От себя замечу, что тестовое задание мне понравилось - достаточно интересное и хитрое